In [1]:
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import Callback

import matplotlib 
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import numpy as np
from time import time
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# Restricting GPU memory
os.environ['CUDA_VISIBLE_DEVICES']='0'
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [3]:
import dataloader, TNN, evaluation # Loading helper functions

In [4]:
dev2train, test2train = dataloader.creating_id_mapping()

# Getting i-vectors
trn_bl_ivector, trn_bg_ivector, dev_bl_ivector, dev_bg_ivector, tst_ivector = dataloader.get_ivectors()

# Loading labels for task 2
trn_bl_id, trn_bg_id, dev_bl_id, dev_bg_id, tst_id = dataloader.get_spk_ids()
dev_trials_label = np.append(dev_bl_id,dev_bg_id)

# Making labels for task 1
trn_ivector = np.append(trn_bl_ivector, trn_bg_ivector,axis=0) # combining bg and bl speakers into a single vector
trn_trials = np.append( np.ones([len(trn_bl_ivector), 1]), np.zeros([len(trn_bg_ivector), 1]))
dev_ivector = np.append(dev_bl_ivector, dev_bg_ivector,axis=0) # combining bg and bl
dev_trials = np.append( np.ones([len(dev_bl_id), 1]), np.zeros([len(dev_bg_id), 1]))
tst_trials, tst_trials_label = dataloader.get_tst_trials()

# Models

## TNN model

In [5]:
model, anchor_input, encoded_anchor = TNN.create_TNN()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 600)          0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 600)          0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 600)          0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 600)          360600      anchor_input[0][0]               
                                                                 positive_input[0][0]       

## Cosine Similarity

In [6]:
def cosine_similarity(embedding, bl_ivector_embedding, bl_id):
    spk_mean, spk_mean_label = dataloader.make_spkvec(bl_ivector_embedding,bl_id)
    # This accuracy calcultion is provided by MCE2018
    scores = spk_mean.dot(embedding.transpose())
    blscores = spk_mean.dot(bl_ivector_embedding.transpose()) # This will be used in normalization and task 2
    
#     # Multi-target normalization
#     mnorm_mu = np.mean(blscores,axis=1) 
#     mnorm_std = np.std(blscores,axis=1)
#     for iter in range(np.shape(scores)[1]):
#         scores[:,iter]= (scores[:,iter] - mnorm_mu) / mnorm_std
    pred_scores = np.max(scores,axis=0)
    
    # Evaluating Task 2
    identified_label = spk_mean_label[np.argmax(scores,axis=0)]
#     tst_trials_confusion = dataloader.get_trials_label_with_confusion(identified_label, bl_id, id_dict, tst_trials )

    return pred_scores,identified_label

# Training Phase

In [7]:
adam_optim = Adam(lr=5e-6)
model.compile(loss=TNN.triplet_loss, optimizer=adam_optim)

In [8]:
# Preparing the data for triplet mining
all_data = np.column_stack((trn_bl_ivector.reshape(3631,3,600),dev_bl_ivector.reshape(3631,1,600)))
# shape = (classes, number of samples per class, feature size)

# Preparing label for the triplet mining
label_ls = []
for i in range(3631):
    for label in range(4):
        label_ls.append(i)
label_array = np.array(label_ls)

In [9]:
# Defining early stopping so that we mine new triplets when the loss is below 1e-6
# The code is from https://stackoverflow.com/questions/37293642
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True
            
callbacks = [EarlyStoppingByLossVal(monitor='loss', value=1e-6, verbose=1)]

In [10]:
total_loss_history = []
total_valloss_history = []

# getting dev ids to train id conversion
dev_bl_id_along_trnset = []
for iter in range(len(dev_bl_id)):
    dev_bl_id_along_trnset.extend([dev2train[dev_bl_id[iter]]])

for i in range(200):    
    print('Mining for {}th times'.format(i+1))
    X_train, X_test = TNN.generate_triplet(all_data.reshape(-1,600),label_array,testsize=0.3,ap_pairs=6,an_pairs=50)    
    Anchor = X_train[:,0,:]
    Positive = X_train[:,1,:]
    Negative = X_train[:,2,:]
    Anchor_test = X_test[:,0,:]
    Positive_test = X_test[:,1,:]
    Negative_test = X_test[:,2,:]
    Y_dummy = np.empty((Anchor.shape[0],1))
    Y_dummy2 = np.empty((Anchor_test.shape[0],1))
    history = model.fit([Anchor,Positive,Negative],y=Y_dummy,validation_data=([Anchor_test,Positive_test,Negative_test],Y_dummy2),
                        batch_size=512, epochs=10, callbacks=callbacks)
    # record down the training history
    total_loss_history = total_loss_history + history.history['loss']
    total_valloss_history = total_valloss_history + history.history['val_loss']
    
    # Making prediction to check the model accuracy
    trained_model = Model(inputs=anchor_input, outputs=encoded_anchor)

    transformed_trn_bl_ivector = trained_model.predict(trn_bl_ivector)
    transformed_dev_bl_ivector = trained_model.predict(dev_bl_ivector)
    combined_embedding = np.append(transformed_trn_bl_ivector,transformed_dev_bl_ivector,0)
    combined_labels = np.append(trn_bl_id,dev_bl_id_along_trnset)
    transformed_tst_ivector = trained_model.predict(tst_ivector)

    
    # Task 1
    task1_pred, task2_pred = cosine_similarity(transformed_tst_ivector, combined_embedding, combined_labels)
    err = dataloader.calculate_EER(tst_trials,task1_pred)

    # Task 2
    tst_trials_confusion = dataloader.get_trials_label_with_confusion(task2_pred, tst_trials_label, test2train, tst_trials )
    dev_EER_confusion = dataloader.calculate_EER_with_confusion(task1_pred,tst_trials_confusion)

Mining for 1th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 13us/step - loss: 0.7751 - val_loss: 0.6655
Epoch 2/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.5164 - val_loss: 0.4088
Epoch 3/10
726200/726200 [==============================] - 9s 13us/step - loss: 0.3271 - val_loss: 0.2946
Epoch 4/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.2452 - val_loss: 0.2391
Epoch 5/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.1983 - val_loss: 0.2030
Epoch 6/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.1651 - val_loss: 0.1760
Epoch 7/10
726200/726200 [==============================] - 9s 13us/step - loss: 0.1394 - val_loss: 0.1547
Epoch 8/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.1187 - val_loss: 0.1373
Epoch 9/10
726200/726200 [==============================] - 9s 13us/ste

726200/726200 [==============================] - 9s 12us/step - loss: 0.0021 - val_loss: 0.0032
Epoch 8/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.0019 - val_loss: 0.0032
Epoch 9/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.0018 - val_loss: 0.0031
Epoch 10/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.0016 - val_loss: 0.0030
Top S detector EER is 10.31%
Top 1 detector EER is 15.31% (Total confusion error is 398)
Mining for 8th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.0026 - val_loss: 0.0027
Epoch 2/10
726200/726200 [==============================] - 9s 13us/step - loss: 0.0024 - val_loss: 0.0026
Epoch 3/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.0022 - val_loss: 0.0026
Epoch 4/10
726200/726200 [==============================] - 8s 12us/step - loss: 0.0020 - val_loss:

726200/726200 [==============================] - 9s 13us/step - loss: 8.5813e-04 - val_loss: 7.9437e-04
Epoch 2/10
726200/726200 [==============================] - 9s 12us/step - loss: 7.5914e-04 - val_loss: 7.7075e-04
Epoch 3/10
726200/726200 [==============================] - 10s 13us/step - loss: 6.7853e-04 - val_loss: 7.4909e-04
Epoch 4/10
726200/726200 [==============================] - 9s 12us/step - loss: 6.0641e-04 - val_loss: 7.2973e-04
Epoch 5/10
726200/726200 [==============================] - 9s 12us/step - loss: 5.4139e-04 - val_loss: 7.1169e-04
Epoch 6/10
726200/726200 [==============================] - 9s 13us/step - loss: 4.8189e-04 - val_loss: 6.9483e-04
Epoch 7/10
726200/726200 [==============================] - 9s 12us/step - loss: 4.2762e-04 - val_loss: 6.7961e-04
Epoch 8/10
726200/726200 [==============================] - 9s 13us/step - loss: 3.7824e-04 - val_loss: 6.6533e-04
Epoch 9/10
726200/726200 [==============================] - 9s 12us/step - loss: 3.3387e-0

726200/726200 [==============================] - 9s 13us/step - loss: 3.3113e-04 - val_loss: 4.4976e-04
Epoch 4/10
726200/726200 [==============================] - 9s 12us/step - loss: 2.8829e-04 - val_loss: 4.3480e-04
Epoch 5/10
726200/726200 [==============================] - 9s 13us/step - loss: 2.5031e-04 - val_loss: 4.2111e-04
Epoch 6/10
726200/726200 [==============================] - 9s 13us/step - loss: 2.1596e-04 - val_loss: 4.0815e-04
Epoch 7/10
726200/726200 [==============================] - 9s 13us/step - loss: 1.8417e-04 - val_loss: 3.9603e-04
Epoch 8/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.5540e-04 - val_loss: 3.8492e-04
Epoch 9/10
726200/726200 [==============================] - 9s 13us/step - loss: 1.3020e-04 - val_loss: 3.7508e-04
Epoch 10/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.0761e-04 - val_loss: 3.6599e-04
Top S detector EER is 10.04%
Top 1 detector EER is 13.80% (Total confusion error is 339)
M

726200/726200 [==============================] - 9s 12us/step - loss: 1.5368e-04 - val_loss: 2.7332e-04
Epoch 6/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.2641e-04 - val_loss: 2.6350e-04
Epoch 7/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.0239e-04 - val_loss: 2.5518e-04
Epoch 8/10
726200/726200 [==============================] - 9s 12us/step - loss: 8.2633e-05 - val_loss: 2.4805e-04
Epoch 9/10
726200/726200 [==============================] - 9s 12us/step - loss: 6.5392e-05 - val_loss: 2.4191e-04
Epoch 10/10
726200/726200 [==============================] - 9s 12us/step - loss: 5.0422e-05 - val_loss: 2.3606e-04
Top S detector EER is 9.97%
Top 1 detector EER is 13.61% (Total confusion error is 332)
Mining for 27th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 13us/step - loss: 2.7777e-04 - val_loss: 2.8165e-04
Epoch 2/10
726200/726200 [=================

...

726200/726200 [==============================] - 9s 12us/step - loss: 5.2218e-05 - val_loss: 4.4297e-05
Epoch 2/10
726200/726200 [==============================] - 9s 13us/step - loss: 2.9777e-05 - val_loss: 4.0622e-05
Epoch 3/10
726200/726200 [==============================] - 9s 13us/step - loss: 1.9416e-05 - val_loss: 3.8053e-05
Epoch 4/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.1823e-05 - val_loss: 3.6645e-05
Epoch 5/10
726200/726200 [==============================] - 9s 12us/step - loss: 6.8162e-06 - val_loss: 3.5834e-05
Epoch 6/10
726200/726200 [==============================] - 9s 12us/step - loss: 3.2093e-06 - val_loss: 3.5042e-05
Epoch 7/10
726200/726200 [==============================] - 9s 12us/step - loss: 8.5643e-07 - val_loss: 3.4748e-05
Epoch 00006: early stopping THR
Top S detector EER is 10.15%
Top 1 detector EER is 13.44% (Total confusion error is 307)
Mining for 98th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10


726200/726200 [==============================] - 9s 13us/step - loss: 1.0207e-05 - val_loss: 3.7664e-05
Epoch 5/10
726200/726200 [==============================] - 9s 13us/step - loss: 4.8076e-06 - val_loss: 3.5677e-05
Epoch 6/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.6275e-06 - val_loss: 3.4809e-05
Epoch 7/10
726200/726200 [==============================] - 9s 13us/step - loss: 4.9998e-07 - val_loss: 3.4537e-05
Epoch 00006: early stopping THR
Top S detector EER is 10.12%
Top 1 detector EER is 13.33% (Total confusion error is 305)
Mining for 105th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 12us/step - loss: 5.9911e-05 - val_loss: 6.2684e-05
Epoch 2/10
726200/726200 [==============================] - 9s 13us/step - loss: 3.5923e-05 - val_loss: 5.8263e-05
Epoch 3/10
726200/726200 [==============================] - 9s 12us/step - loss: 2.4013e-05 - val_loss: 5.5283e-05
Epoch 4/10

726200/726200 [==============================] - 9s 12us/step - loss: 3.8364e-06 - val_loss: 2.6233e-05
Epoch 6/10
726200/726200 [==============================] - 9s 12us/step - loss: 2.0055e-06 - val_loss: 2.5964e-05
Epoch 7/10
726200/726200 [==============================] - 9s 12us/step - loss: 7.2130e-07 - val_loss: 2.5772e-05
Epoch 00006: early stopping THR
Top S detector EER is 10.10%
Top 1 detector EER is 13.41% (Total confusion error is 302)
Mining for 113th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 13us/step - loss: 3.9753e-05 - val_loss: 3.5000e-05
Epoch 2/10
726200/726200 [==============================] - 9s 13us/step - loss: 1.8864e-05 - val_loss: 3.2458e-05
Epoch 3/10
726200/726200 [==============================] - 9s 13us/step - loss: 1.0525e-05 - val_loss: 3.0534e-05
Epoch 4/10
726200/726200 [==============================] - 9s 13us/step - loss: 6.2098e-06 - val_loss: 2.9129e-05
Epoch 5/10

726200/726200 [==============================] - 9s 12us/step - loss: 8.5094e-06 - val_loss: 2.8812e-05
Epoch 5/10
726200/726200 [==============================] - 9s 12us/step - loss: 3.6881e-06 - val_loss: 2.7373e-05
Epoch 6/10
726200/726200 [==============================] - 9s 13us/step - loss: 1.1387e-06 - val_loss: 2.6608e-05
Epoch 7/10
726200/726200 [==============================] - 9s 13us/step - loss: 2.1952e-07 - val_loss: 2.5769e-05
Epoch 00006: early stopping THR
Top S detector EER is 9.98%
Top 1 detector EER is 13.44% (Total confusion error is 311)
Mining for 121th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 13us/step - loss: 4.4748e-05 - val_loss: 4.4114e-05
Epoch 2/10
726200/726200 [==============================] - 9s 13us/step - loss: 2.4651e-05 - val_loss: 4.0819e-05
Epoch 3/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.5180e-05 - val_loss: 3.9258e-05
Epoch 4/10


KeyboardInterrupt: 